# Data Cleaning and Preparation

In [139]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data
- Missing data occurs commonly in many data analysis applications
- One of the goals of pandas is to make working with missing data as painless as possible
    - E.g.: all of the descriptive statistics on pandas objects exclude missing data by default

- For numeric data: the floating-point value `NaN` (Not a Number) represents missing data
    - sentinel value that can be easily detected

In [140]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

- The built-in Python None value is also treated as NA in object arrays

In [141]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In [142]:
print(string_data[0])
print(type(string_data[0]))
print()
print(string_data[2])
print(type(string_data[2]))

None
<class 'NoneType'>

nan
<class 'float'>


### Filtering Out Missing Data
- As a convention, we use `NA` (for Not Available) to indicate the missing values
    - `na` is often in the name of the functino we might looking for when dealing with missing values in Pandas
    - other time we'll find `null`

In [143]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [144]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [145]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


#### `dropna`
- drop the entire rwo that contains a NA

In [146]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


- Passing how='all' will only drop rows that are all NA

In [147]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [148]:
# make the 3rd row of all NA
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [149]:
# drop the column with all NA
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [150]:
# by defaulr, axis=0, meaning "row"
data.dropna(how='all')

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [151]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [152]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


- if we can tollerate a threshold for the number of missing values in a row, we can specify that

In [153]:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling In Missing Data
- Pandas comes with a series of method for filling missing data

In [154]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


- we can specify what value to use for each colum passing a dict

In [155]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


- as always, we can perform the operation `inplace` (dafault is always Fase)

In [156]:
df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [157]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [158]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


In [159]:
df.fillna(df.mean())

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,1.686533,1.343810
3,-0.713544,1.686533,-2.370232
4,-1.860761,1.686533,-0.436259
5,-1.265934,1.686533,-0.436259


In [160]:
# remember: man of the value for each column
df.mean()
# df.mean(axis=1)

0   -0.569428
1    1.686533
2   -0.436259
dtype: float64

## Data Transformation

### Removing Duplicates

In [161]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [162]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

#### `drop_duplicates`
- remove duplicated rows

In [163]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [164]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


- duplicates of a specific subset of the row

In [165]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [166]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [167]:
data.drop_duplicates(['k1', 'k2'], keep='first')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


### Transforming Data Using a Function or Mapping
- For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame

In [168]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [169]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [170]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [171]:
lowercased.map(lambda x: meat_to_animal.get(x))

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

- the `map` function accept also directly a `dict` 
    - implicitly pass the labda function as above

In [172]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [173]:
# one line
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


- Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations

### Replacing Values
- many times we want to replace values
- example: disguised missgin values
    - values that should be null but are not null

In [174]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [175]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [176]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [177]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [178]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes
- Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects
- We can also modify the axes in-place without creating a new data structure

In [179]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [180]:
transform = lambda x: x[:4].upper() # truncate to the 4th character and make uppercase
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [181]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


- `rename` 
    - apply a function to index and/or columns
    - apply a dict conversion

In [182]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [183]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning
- Continuous data is often discretized or otherwise separated into “bins” for analysis
- Suppose we have data about a group of people in a study, and you want to group them into discrete age buckets

In [184]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [187]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [193]:
cats[0]

Interval(18, 25, closed='right')

#### codes
- returns the array of `ages` transformed replacing each age with the bin-id to which it correspond
- e.g.: it it's 20, it's between (18,25], hence its bin-id is the first one, i.e., `0`

In [195]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [196]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

#### values_conunts
- convienient for histograms
    - how many element for each bin?

In [198]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

#### closed and open ranges
- `(18` means 18 is excluded
- `[18` means 18 is included

- a parenthesis means that the side is open, while the square bracket means it is closed (inclusive
- we can change which side is closed by passing right=False

In [204]:
cats2 = pd.cut(ages, [18, 26, 36, 61, 100], right=False)
cats2

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

#### intervals
- bins are represented as `interval`
- we can check for overlaps

In [219]:
print(cats2.categories[0])
print(cats.categories[1])
cats2.categories[0].overlaps(cats.categories[1])

[18, 26)
(25, 35]


True

#### naming
- we can name each bin

In [220]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

#### cutting float array
- we can specify the precision

In [222]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.11, 0.32], (0.74, 0.95], (0.32, 0.53], (0.74, 0.95], (0.74, 0.95], ..., (0.74, 0.95], (0.74, 0.95], (0.74, 0.95], (0.11, 0.32], (0.74, 0.95]]
Length: 20
Categories (4, interval[float64, right]): [(0.11, 0.32] < (0.32, 0.53] < (0.53, 0.74] < (0.74, 0.95]]

#### quantiles
- we can even cut by quantiles

In [225]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats
pd.value_counts(cats)

(-3.746, -0.609]     250
(-0.609, -0.0106]    250
(-0.0106, 0.681]     250
(0.681, 3.26]        250
dtype: int64

In [233]:
cq = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
# cq = pd.qcut(data, [0, 0.1, 0.2, 1.])
cq

[(-1.299, -0.0106], (-0.0106, 1.338], (-1.299, -0.0106], (-1.299, -0.0106], (-0.0106, 1.338], ..., (-3.746, -1.299], (1.338, 3.26], (-0.0106, 1.338], (-0.0106, 1.338], (-1.299, -0.0106]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.746, -1.299] < (-1.299, -0.0106] < (-0.0106, 1.338] < (1.338, 3.26]]

In [234]:
pd.value_counts(cq)

(-1.299, -0.0106]    400
(-0.0106, 1.338]     400
(-3.746, -1.299]     100
(1.338, 3.26]        100
dtype: int64

### Detecting and Filtering Outliers

In [243]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.002605,0.012872,-0.025054,-0.006163
std,1.028576,0.993650,1.007879,0.965846
min,-2.999799,-2.938085,-3.255825,-3.398085
25%,-0.686750,-0.691499,-0.686546,-0.630043
50%,0.016320,0.031010,0.026978,0.028120
75%,0.670908,0.683887,0.665875,0.640438
max,3.144389,3.003284,3.897527,3.521136


- Suppose you wanted to find values in one of the columns exceeding 3 in absolute value
    - which is ~ 3std. dev.

In [244]:
col = data[2]
col[np.abs(col) > 3]

193    3.061029
534    3.897527
562    3.048747
876   -3.255825
Name: 2, dtype: float64

- To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame

In [258]:
# any tests whether any array element along a given axis evaluates to True
data[(np.abs(data) > 3).any(True)]

,0,1,2,3
193,1.490529,-0.928871,3.061029,-0.297909
269,3.144389,0.391328,0.672451,-1.571846
534,-0.923066,-1.554274,3.897527,0.677627
552,-2.878581,0.457118,1.638731,3.160760
562,0.706299,2.133070,3.048747,0.264626
663,-1.361069,3.003284,2.151570,-0.414736
761,1.271806,-0.229430,1.035012,-3.398085
810,-0.687130,0.295366,-1.448936,3.484365
848,-0.394359,1.705631,0.928319,3.521136
876,-1.916714,0.799651,-3.255825,0.213170


- Values can be set based on these criteria
- E.g., we can cap the values outside the interval –3 to 3

In [259]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.002750,0.012869,-0.025805,-0.006931
std,1.028144,0.993640,1.003652,0.960605
min,-2.999799,-2.938085,-3.000000,-3.000000
25%,-0.686750,-0.691499,-0.686546,-0.630043
50%,0.016320,0.031010,0.026978,0.028120
75%,0.670908,0.683887,0.665875,0.640438
max,3.000000,3.000000,3.000000,3.000000


In [260]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,-1.0,1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,1.0
3,1.0,1.0,1.0,1.0
4,1.0,-1.0,-1.0,-1.0


### Permutation and Random Sampling
Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the `numpy.random.permutation` function
- Calling permutation with the length of the axis we want to permute produces an array of integers indicating the new ordering
- Many Machine Learning pipelines require that

In [265]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([4, 0, 3, 1, 2])

In [266]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [267]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11


In [268]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [269]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

3    6
2   -1
0    5
2   -1
1    7
2   -1
3    6
3    6
0    5
1    7
dtype: int64

### Computing Indicator/Dummy Variables
- converting a categorical variable into a “dummy” matrix (see also one-hot-encoding)
- useful for statistical modeling or machine learning applications

How it works
- If a column in a DataFrame has k distinct values, we would derive a matrix or DataFrame with k columns containing all 1s and 0s
- pandas has a get_dummies function for doing this

In [278]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [279]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


- specify the column directly

In [280]:
df = pd.DataFrame([('Foreign Cinema', 289.0),
                   ('Liho Liho', 224.0),
                   ('500 Club', 80.5),
                   ('Foreign Cinema', 25.30)],
           columns=('name', 'Amount')
                 )

df

,name,Amount
0,Foreign Cinema,289.0
1,Liho Liho,224.0
2,500 Club,80.5
3,Foreign Cinema,25.3


In [281]:
pd.get_dummies(df, columns=['name'])

,Amount,name_500 Club,name_Foreign Cinema,name_Liho Liho
0,289.0,0,1,0
1,224.0,0,0,1
2,80.5,1,0,0
3,25.3,0,1,0


## String Manipulation

### String Object Methods

In [282]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [283]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [284]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [285]:
'::'.join(pieces)

'a::b::guido'

In [286]:
'guido' in val
val.index(',')
val.find(':')

-1

In [288]:
val.count(',')

2

In [290]:
print(val.replace(',', '::'))
print(val.replace(',', ''))

a::b::  guido
ab  guido


### Regular Expressions
- Regular expressions provide a flexible way to search or match (often more complex) string patterns in text
- A single expression (called a **regex**) is a string formed according to the regular expression language
- Python’s built-in `re` module is responsible for applying regular expressions to strings

regex are a powerful tool, but it might take some time to master it
- here we just explore the basics

In [300]:
import re
text = "foo    bar\t baz  \tqux"

# one or more whitespace characters is \s+
# \s is the space
# + means one or more
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [301]:
# find all the occurrences that match the pattern
re.findall('\s+',text)

['    ', '\t ', '  \t']

- for instance, we can write a regex to extract all the eamil from a text

In [312]:
text = """Dave's mail is dave@google.com 
Steve's is steve@gmail.com
Rob's one instead is rob@gmail.com
finally Ryan has ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [313]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [314]:
m = regex.search(text)
m
text[m.start():m.end()]

'dave@google.com'

In [315]:
print(regex.match(text))

None


In [316]:
print(regex.sub('REDACTED', text))

Dave's mail is REDACTED 
Steve's is REDACTED
Rob's one instead is REDACTED
finally Ryan has REDACTED

